<a href="https://colab.research.google.com/github/shannonfernandes25/bioinformatics-BPRI-Bioinformatics/blob/main/positive_embeddings_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers tqdm biopython torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.1 MB/s eta 0:00:00


In [2]:
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModel

In [3]:
# Load your data
df = pd.read_excel("/content/positive_dataset.xlsx")

# Ensure 'Sequence' column exists and converts to list
sequences = df["sequence"].tolist()

In [4]:
df['sequence'] = df['sequence'].astype(str).str.strip()  #str.strip() removes leading and trailing spaces from each string value
df = df[df['sequence'].str.len() > 0]


df = df[~df['sequence'].str.lower().isin(['nan', 'none', 'null'])]
#str.lower() converts all sequence entries to lowercase for consistent comparison.
#The tilde ~ negates the mask   Goal: remove textual missing-value markers like "nan", "None", or "null".

# Final list of sequences
sequences = df['sequence'].tolist()

In [5]:
# generate mean embeddings
def generate_embeddings(model_name, sequences, device):
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    embeddings = []
    with torch.no_grad():
        for seq in tqdm(sequences):
            # Tokenize and convert to tensor
            tokens = tokenizer(seq, return_tensors='pt', truncation=True, max_length=1024)
            tokens = {k: v.to(device) for k, v in tokens.items()}

            outputs = model(**tokens)
            # Take mean across all residues
            emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            embeddings.append(emb)
    return embeddings

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Generate embeddings for each model
models = {
    "ProtBERT03": "Rostlab/prot_bert_bfd",
    "ProtBERT05": "Rostlab/prot_bert",
    "ESM_t6": "facebook/esm2_t6_8M_UR50D",
    "ESM_t12": "facebook/esm2_t12_35M_UR50D"
}

# Generate and store
all_embeddings = {}
for label, model_name in models.items():
    print(f"\n🔹 Generating {label} embeddings...")
    all_embeddings[label] = generate_embeddings(model_name, sequences, device)

# Combine all embeddings into one DataFrame
for label in all_embeddings:
    emb_df = pd.DataFrame(all_embeddings[label])
    emb_df.columns = [f"{label}_{i}" for i in range(emb_df.shape[1])]
    df = pd.concat([df, emb_df], axis=1)

# Save to CSV
output_path = "positive_protein_embeddings.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ Embeddings saved to: {output_path}")


🔹 Generating ProtBERT03 embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/136M [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 886/886 [00:39<00:00, 22.42it/s]



✅ Embeddings saved to: positive_protein_embeddings.csv
